<a href="https://colab.research.google.com/github/arind123/GenAI/blob/main/6-RentalGenerator_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip install textblob 'keras-nlp' 'keras-preprocessing' 'gensim==4.2.0' np_utils swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.5/584.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.7 MB/s eta 0:00:00
  Created wheel for np_utils: f

In [3]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda, ELU, Conv1D, MaxPooling1D, Dropout, GRU, LSTM
from keras import Model, Input
import np_utils
import swifter
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from tensorflow.keras.layers.experimental import preprocessing
import keras_nlp
import os
import time
import sys
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk
import pickle
from tensorflow.nn import leaky_relu

import re
import warnings
from sklearn.model_selection import train_test_split
from textblob import TextBlob
from collections import defaultdict


TRACE = False
embedding_dim = 100
rnn_units = 128
epochs=100
buffer_size = 256
corpus_size=1000
test_corpus_size=50
# Batch size
batch_size = 256
min_count_words = 3
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

Using TensorFlow backend


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
%%writefile get_data.sh
if [ ! -f train_corpus_descriptions_airbnb.csv ]; then
  wget -O train_corpus_descriptions_airbnb.csv https://www.dropbox.com/s/5rp7ibop99qyafo/train_corpus_descriptions_airbnb.csv?dl=0
fi

if [ ! -f test_corpus_descriptions_airbnb.csv ]; then
    wget -O test_corpus_descriptions_airbnb.csv https://www.dropbox.com/s/a29bbkg8hi4q4f4/test_corpus_descriptions_airbnb.csv?dl=0
fi

Writing get_data.sh


In [5]:
!bash get_data.sh

--2023-11-21 14:40:46--  https://www.dropbox.com/s/5rp7ibop99qyafo/train_corpus_descriptions_airbnb.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/5rp7ibop99qyafo/train_corpus_descriptions_airbnb.csv [following]
--2023-11-21 14:40:46--  https://www.dropbox.com/s/raw/5rp7ibop99qyafo/train_corpus_descriptions_airbnb.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4584a88e60d8480e1006d1fa1d.dl.dropboxusercontent.com/cd/0/inline/CH8Y4mBzIcI53w5YxZjh4cBReBN6cgOmiCqYlHhl0Uu9_BpcLmSGzRzD-9HT1QioUkaaE3kziJFlYCtnZ4U88jYbVeA6in_WRDZDK7RJjos0S94SQ-ggHprUoMIwti7lJCbYZpdoyh8-AgVQYmiTosyC/file# [following]
--2023-11-21 14:40:46--  https://uc4584a88e60d8480e1006d1fa1d.dl.dropboxusercontent.com/cd/0/inline/CH8Y4mBzIcI53w5YxZjh4cBRe

In [6]:
train_path = "./train_corpus_descriptions_airbnb.csv"
test_path = "./test_corpus_descriptions_airbnb.csv"
# Read, then decode for py2 compat.
airbnb_reviews = pd.read_csv(train_path, header=None, names=["review"]).dropna().sample(n=corpus_size).reset_index(drop=True)
test_airbnb_reviews = pd.read_csv(test_path, header=None, names=["review"]).dropna().sample(n=test_corpus_size).reset_index(drop=True)


In [8]:
airbnb_reviews

,review
0,"Comfortable, quiet garden-facing double room w..."
1,Welcome to the Giant Oaks House located just o...
2,The apartment is located just 2 minutes walk f...
3,Stay for 90+ nights (minimum nights and rates ...
4,"Ideal place for tourists , visiting los angele..."
...,...
995,"Quiet, clean 3-bedroom apartment in the heart ..."
996,Hello potential guests!My house is a newly bui...
997,"My place is close to Haggerston Station, Broad..."
998,The flat is spacious and ideally located just ...


In [9]:
# Take a look at the first review in text
print(airbnb_reviews.iloc[0].review)


Comfortable, quiet garden-facing double room with own en-suite (shower & toilet) on the 2nd floor of a lovely two-story 2.5 bedroom home, conveniently located in between 3 tubes stations, plenty of buses, bike hiring , walking distance to Hyde Park and canals. You are welcome to make yourself at home in all common areas including the large lounge, kitchen and communal garden as well as the study if you require a quiet work-desk.The spaceA large, bright, cozy home with extra high ceiling and full length double glazed windows, set over the ground and first floors. Comprising of a spacious carpeted lounge with leather sofas, flat screen TV, DVD player, streaming services, with large double doors opening to a comfortable eat- in kitchen with polished wooden floor and all mod cons. Free internet available on the wireless network which reaches all the way out to the quiet communal garden.The available tranquil bedroom looks out to the communal garde


In [10]:
def preprocess_text(text, should_join=True):
    text = str(text)
    text = ' '.join(str(word).lower() for word in textblob_tokenizer(text))
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

In [12]:
reivew_processed = airbnb_reviews.review.swifter.apply(lambda x: preprocess_text(x))

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [13]:
def get_maximum_review_length(df):
    maximum = 0
    for ix, row in df.iterrows():
        candidate = len(preprocess_text(row.review, should_join=False))
        if candidate > maximum:
            maximum = candidate
    return maximum


maximum = get_maximum_review_length(airbnb_reviews)


In [24]:
vocab_ = list()

# Construct the vocabulary, you are free to iterate the DF or use TF utilities.
# What is key is that the vocab will be of words such that their frequency is above min_count_words!
for i in reivew_processed:
  for word in preprocess_text(i, False):
    vocab_.append(word)

print(len(vocab_))
from collections import Counter
c = Counter(vocab_)
vocab = [k for k, v in c.items() if v > min_count_words]
vocab = list(set(vocab))
print(len(vocab))

114262
2429


In [25]:
print(f'{len(vocab)} unique words')

2429 unique words


In [26]:
# ids_from_words = None  # Construct the ids from words mapping
from tensorflow.keras.layers.experimental import preprocessing
ids_from_words = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)

In [27]:
# words_from_ids = None  # And the inverse
words_from_ids = preprocessing.StringLookup(vocabulary=ids_from_words.get_vocabulary(), invert=True, mask_token=None)

In [28]:
def text_from_ids(ids):
  return tf.strings.reduce_join(words_from_ids(ids), axis=-1, separator=' ')

In [29]:
ids = ids_from_words(preprocess_text('Only you can prevent forest fires', should_join=False))
ids

<tf.Tensor: shape=(6,), dtype=int64, numpy=array([ 675, 1494, 1630,    0,  930,    0])>

In [30]:
text_from_ids(ids)

<tf.Tensor: shape=(), dtype=string, numpy=b'only you can [UNK] forest [UNK]'>

In [52]:
all_ids = tf.constant(np.zeros((1, maximum)), dtype='int64')
review = 'Only you can prevent forest fires'
review = preprocess_text(review, should_join=False) # Preprocess the review (should it output text or a list?)
value = ids_from_words(review[0: maximum])

if len(value) < maximum:
  pad_len = maximum - len(value)
  value = np.pad(value, (0, pad_len), 'constant' )
value = tf.reshape(value, [1, maximum])
output = tf.concat([all_ids, value], 0)
# output = tf.reshape(output, [1, maximum])
output

<tf.Tensor: shape=(2, 178), dtype=int64, numpy=
array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   

In [ ]:
def get_ids_tensor(df):
  all_ids = tf.constant(np.zeros((1, maximum)), dtype='int64')
  for review in df.review:
      review = preprocess_text(review, should_join=False) # Preprocess the review (should it output text or a list?)
      value = ids_from_words(review[0: maximum])
      # value = None  # if the review has fewer words than maximum pad with zeroes on the right
      if len(value)< maximum:
        pad_len = maximum - len(value)
        value = np.pad(value, (0, pad_len), 'constant' )
      value = tf.reshape(value, [1, maximum])
      output = None # concatenate
      all_ids = None  # reshape
  return all_ids[1:]

all_ids = get_ids_tensor(df=airbnb_reviews)
print(all_ids)

In [ ]:
test_all_ids = get_ids_tensor(df=test_airbnb_reviews)

In [ ]:
#Prepare the dataset
ids_dataset = None # Create tensorflow Dataset object from all_ids tensor
test_ids_dataset = None # Do likewise for test set

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = ids_dataset.map(split_input_target)
test_dataset = test_ids_dataset.map(split_input_target)


In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

In [ ]:
dataset = (
    dataset
    .shuffle(buffer_size)
    .batch(batch_size=batch_size, drop_remainder=True)
)
test_dataset = (
    test_dataset
    .batch(batch_size=batch_size, drop_remainder=True)
)

In [ ]:
dataset.take(1)

In [ ]:
class RentalGenerator(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units, seq_length):
    super().__init__(self)
    self.vocab_size = vocab_size
    self.seq_length = seq_length
    self.embedding = None # Create Embedding layer, which should be its shape? If you want test out training a word2vec and set it here to fine-tune
    self.rnn = None # Create a LSTM layer with tanh activation to ensure you can use cuDNN libraries. You should return both sequences and states
    self.dense = None # Final Dense layer as always

  def call(self, inputs, states=None, return_state=False, training=False):
    # Implement the forward pass
    pass

  def build_graph(self):
    x = Input(shape=(self.seq_length, ))   # doesn't consider the batch
    Model(inputs=x, outputs=self.call(x))
    self.build((None, self.seq_length, ))   # takes into consideration the batch size

In [ ]:
model = RentalGenerator(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_words.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    seq_length=seq_length)

In [ ]:
model.build_graph()

In [ ]:
model.summary()

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
# Compile and train the model such that it early stops if the perplexity in the val set does not decrease

In [ ]:
import matplotlib.pyplot as plt

# function for plotting loss
def plot_metrics(train_metric, val_metric=None, metric_name=None, title=None, ylim=5):
    plt.title(title)
    plt.ylim(0,ylim)
    plt.plot(train_metric,color='blue',label=metric_name)
    if val_metric is not None: plt.plot(val_metric,color='green',label='val_' + metric_name)
    plt.legend(loc="upper right")

# plot loss history
plot_metrics(history.history['loss'], val_metric=history.history['val_loss'], metric_name="Loss", title="Loss", ylim=5.0)

In [ ]:
plot_metrics(history.history['perplexity'], val_metric=history.history['val_perplexity'], metric_name="perplexity", title="perplexity", ylim=2000.0)

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, word_from_ids, ids_from_words, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.word_from_ids = word_from_ids
    self.ids_from_words = ids_from_words

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_words(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_words.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function(reduce_retracing=True)
  def expand_dims_if_neccesary(self, input):
    if len(input.shape) < 3:
      input = tf.expand_dims(input, axis=0)
    return input

  @tf.function(reduce_retracing=True)
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_words = preprocess_text(inputs, should_join=False)
    input_ids = self.expand_dims_if_neccesary(self.ids_from_words(input_words))
    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_word = self.word_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_word, states

In [ ]:
one_step_model = OneStep(model, words_from_ids, ids_from_words)
start = time.time()
states = None
description = tf.constant(['Midtown Sunny 2-Bedroom'])
# result = [description]

for n in range(100):
  next_word, states = one_step_model.generate_one_step(description, states=states)
  description = tf.concat([description, next_word], axis=0)


result = tf.strings.join(description, separator=" ")
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)



In [ ]:
tf.saved_model.save(one_step_model, 'lstm_rental_generator')